# Combination Locks

This is a solution to the OOP Exercise described in: 

https://gist.github.com/ljbelenky/845ceb92207ab3b8b69697538575e2f6


## Note:

In this version, if verbose is true, it will attempt to send a text message to a cell phone if the lock is opened.

This functionality relies on a free trial service from Twilio, as described here:

https://www.fullstackpython.com/blog/send-sms-text-messages-python.html

Besides the twilio REST API, which can be installed with  `pip install twilio`, this requires a Twilio account, an SID, a token and a verified cell-phone number.

Because these values should not be put into a GitHub repo, I have put them in my `~/.bashrc` as environment variables using:

```
export TWILIO_SID=AC99999999999999999999999999999999
export TWILIO_TOKEN=d9999999999999999999999999999999

export TWILIO_TO=+12125551212
export TWILIO_FROM=+12515551212
```

To enable this functionality, you'll need to acquire your own SID and TOKEN. Don't forget to `source ~/.bashrc` to activate any changes.

### Defintion of ComboLock Class

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from twilio.rest import Client

class ComboLock:
    def __init__(self, digits = 4):
        self.digits = digits
        
        # Pick a random code number
        combo = np.random.randint(0,10, digits)
        
        # Hash the code to hide it from prying eyes
        self._hashed_combo = hash(str(combo))
        
        self.clear()        
        
    def clear(self):
        self._input = []
        
    def try_to_open(self, verbose = False):
        '''
        Returns True if the correct combination has been entered.
        If verbose is true, prints output and sends text message to cell phone
        '''
        
        if hash(str(np.array(self._input))) == self._hashed_combo:
            if verbose:
                print(f'You found the correct combo: {self._input}')
                try:
                    sid = os.environ['TWILIO_SID']
                    token = os.environ['TWILIO_TOKEN']
                    to = os.environ['TWILIO_TO']
                    from_ = os.environ['TWILIO_FROM']
                    
                    message = f'You opened the lock with combination {self._input}'
        
                    client = Client(sid, token)
                    client.messages.create(to=to,
                                          from_=from_,
                                          body= message)
                except Exception as E:
                    print(E)
            return True
        return False
    
    def input_digit(self, d):
        
        try:
            # Only accept the first digit of input
            d = int(str(d)[0])
        
            # Only accept valid input digits
            if d in range(0,10):
                self._input.append(d)        
        except:
            print('Invalid Input')
        
    def __repr__(self):
        state = 'unlocked' if self.try_to_open() else 'locked'
        return f'{state} ComboLock with {self.digits}-digit combination'

### Attempt to guess the combination

In [ ]:
my_lock = ComboLock()

attempts = 0
while True:
    attempts += 1
    my_lock.clear()
    for i in range(my_lock.digits):
        guess = np.random.randint(0,10)
        my_lock.input_digit(guess)
    
    if my_lock.try_to_open(True):
        print(f'In {attempts} attempts')
        break

### Make a function to guess the combination randomly:

In [ ]:
def random_guess(lock):
    attempts = 0
    while True:
        attempts += 1
        my_lock.clear()
        for i in range(my_lock.digits):
            guess = np.random.randint(0,10)
            my_lock.input_digit(guess)
        if my_lock.try_to_open():
            return attempts

### Make a bunch of locks and see how how many trys it takes to unlock them

In [ ]:
locks = [ComboLock() for _ in range(100)]
random_results = list(map(random_guess, locks))
plt.hist(random_results)
plt.xlabel('Number of attempts until lock opens')
plt.ylabel('Number of locks that open with this number of attempts');
print(np.mean(random_results))

#### What's the shape of this distribution?

### Let's try a more organized approach:

In [ ]:
def ordered_guess(lock):
    attempts = 0
    for i in range(10**(lock.digits)):
        code = str(i).zfill(lock.digits)
        lock.clear()
        attempts +=1
        for d in code:
            lock.input_digit(int(d))
        if lock.try_to_open():
            return attempts 

### And see how long it takes to open the locks this way

In [ ]:
ordered_results = list(map(ordered_guess, locks))
plt.hist(ordered_results)
plt.xlabel('Number of attempts until lock opens')
plt.ylabel('Number of locks that open with this number of attempts');

print(np.mean(ordered_results))

#### What's the shape of this distribution?

In [ ]:
def cdf(value, array):
    return (array<value).sum()/len(array)

def vcdf(dist):
    dist = np.array(dist)
    return np.vectorize(cdf, excluded = ['array'])(value = dist, array=dist)

### Let's compare the distributions of the two different approaches:

In [ ]:
plt.scatter(random_results, vcdf(random_results), marker = '.', label = 'Random')
plt.scatter(ordered_results, vcdf(ordered_results), marker = '.', label = 'Ordered')
plt.xlabel('Number of attempts until lock opens')
plt.ylabel('Cumulative Distribution')
plt.legend();


### Is one approach better than the other?  Why or why not?